In [1]:
import os

In [2]:
%pwd

'c:\\Users\\cogni\\Downloads\\data-science-projects\\mlops-end-to-end\\research'

In [3]:
os.chdir("..")

In [4]:
%pwd

'c:\\Users\\cogni\\Downloads\\data-science-projects\\mlops-end-to-end'

Config.yaml


`data_transformation:
    root_dir: artifacts/data_transformation
    data_path: artifacts/data_ingestion/winequality-red.csv`

Update entity_config

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

Update Configuration Manager

In [6]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

[2023-12-29 18:59:39,627: INFO: __init__: Logging setup complete]


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

Update Components

In [8]:
from ml_project import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    # You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_splitting as this data is already cleaned up


    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info("Training data dimensions %s", train.shape)
        logger.info("Test data dimensions %s", test.shape)


In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2023-12-29 18:59:41,059: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-29 18:59:41,062: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-29 18:59:41,065: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-29 18:59:41,066: INFO: common: created directory at: artifacts]
[2023-12-29 18:59:41,067: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-29 18:59:41,117: INFO: 850420403: Splited data into training and test sets]
[2023-12-29 18:59:41,119: INFO: 850420403: Training data dimensions (1199, 12)]
[2023-12-29 18:59:41,123: INFO: 850420403: Test data dimensions (400, 12)]
